# Single-source shortest path (SSSP)

*Reading material: [Chapter 8](https://jeffe.cs.illinois.edu/teaching/algorithms/book/08-sssp.pdf) from Erickson's book.*

This is an algorithm that determines the shortest distance between two vertices in an acyclic graph. To ensure there are no cycles, the graph must also be directed. There are ways to deal with undirected graphs and cycles, but for now we'll focus on a *dag*, a **d**irected **a**cyclic **g**raph. Furthermore, we'll consinder a *weighted* dag:
$$
G = (V,E,w)
$$
where $w: V\times V\mapsto \mathbb{N}_0$ maps an edge to an non negative value called the weight of the edge. In practice, all three objects of the triplet $(V, E, w)$ are packed in an the adjacency matrix of a graph. Given an array ``G`` with the adjacency matrix, the objects $V$, $E$, and $w$ are related as:

\begin{align}
 V & = \{ i\ \mid\ 0\leq i < \texttt{len(G)} \} \\
 E & = \{ (u,v)\ \mid\ u, v\in V\ \text{and}\ \texttt{G[u][v]} < \infty\} \\
 w & =  \{ (u,v)\mapsto \texttt{G[u][v]}\ \mid\ (u,v)\in E\}
\end{align}

For example, if

$$
\texttt{G} =
\begin{bmatrix}
\infty &    160 & \infty & \infty &    140 \\
\infty & \infty &     50 & \infty & \infty \\
\infty & \infty & \infty &     40 & \infty \\
\infty & \infty & \infty & \infty & \infty \\
\infty & \infty & \infty &     30 & \infty
\end{bmatrix}
$$

then

\begin{align}
 V & = \{ i\ \mid\ 0\leq i < \texttt{len(G)} \} = \{0,1,2,3,4\} \\
 E & = \{ (u,v)\ \mid\ \texttt{G[u][v]} < \infty\} = \{(0,1), (0,4), (1,2), (2,3), (4,3)\} \\
 w & =  \{ (u,v)\mapsto \texttt{G[u][v]}\ \mid\ (u,v)\in E\} \\
   &= \{(0,1)\mapsto 160,\ (0,4)\mapsto 140,\ (1,2)\mapsto 50,\ (2,3)\mapsto 40,\ (4,3) \mapsto 30\}
\end{align}



The SSSP algorithm finds the shortest distance from a source vertex $s$ to every vertex in a graph.

## Initialization

In this phase we setup three data structures necessary to implement the algorithm.

\begin{align}
\texttt{dist}[v] &: \text{length of shortest path from } s\text{ to } v \\
\texttt{parent}[v] &: \text{parent vertex of } v\text{ in the shortest path }s\rightsquigarrow v
\end{align}

Initially we set $\texttt{dist}[v] = \infty$ for every $v\neq s$ and  $\texttt{dist}[s] = 0$. Also we set $\texttt{parent}[v] = \text{null}$ for every $v\in V$. And we load $s$ into an array that will drive the iterative part of the algorithm.
```python
dist = []
parent = []
for v in range(len(G)):
  dist.append(float('int'))
  parent.append(None)
dist[s] = 0
```

# Iterations

This part of the algorithm continues for as long as there are vertices in array `stack`. As the name implies, we load and remove data from this array in last-in/first-out fashion.

The objective of the iterative part is to find every tense edge in the graph and relax it. An edge $u\rightarrow v$ with weight $w(u,v)$ is tense when
$$
\color{brown}{\texttt{dist}[v]}  > \color{green}{\texttt{dist}[u]} +\color{magenta}{w(u,v)}
$$
What this means is that the shortest path from
$s\color{brown}{\rightsquigarrow \ldots\rightsquigarrow v}$
is *longer* than the shortest path
$s\color{green}{\rightsquigarrow \ldots\rightsquigarrow u}$
plus the edge
$\color{green}u\color{magenta}\rightarrow\color{brown}v$.

![](https://drive.google.com/uc?export=view&id=18uy8w__FSkA3MmWR7OX55pCi7O9d6Iic)

In other words, the green path above plus the edge
$\color{green}u\color{magenta}\rightarrow\color{brown}v$
provides a shorter way to vertex $\color{brown}v$ than the currently *assumed* short path shown in light brown.

**A tense edge $u \rightarrow v$ is an indication that there is a shorter path to $v$ than the current one.** To relax a tense edge, we accept the shortest path it indicates:
\begin{align}
\text{if } \color{brown}{\texttt{dist}[v]}  > \color{green}{\texttt{dist}[u]} +\color{magenta}{w(u,v)} \\
\text{then } \color{brown}{\texttt{dist}[v]}  = \color{green}{\texttt{dist}[u]} +\color{magenta}{w(u,v)}
\end{align}

The greedy strategy above tells us what to do when we find a tense edge. But how can we be sure we have relaxed every tense edge in the graph? Our first thought would be to explore every edge in the graph:



```python
for u = 0 ... n-1:
  for v = 0 ... n-1:
    if G[u][v] < ∞:
      if dist[v] > dist[u] + G[u][v]:
        dist[v] = dist[u] + G[u][v]:
```

And then repeat the exploration above until no more tense edges are found. The loops above require $\mathcal{O}(n^2)$ time; if we repeat this process another $\mathcal{O}(n)$ times, the algorithm enters $\mathcal{O}(n^3)$ territory.

We may speed things up a bit, perhaps returning to $\mathcal{O}(n^2)$, by exploiting the initial state of the algorithm. At that time, with $\texttt{dist}[v] = \infty$ for $v\neq s$ and $\texttt{dist}[s] = 0$, every edge from $s$ to its neighbors, is a tense one.

Let's assume that the neighbors of $s$ are vertices $s_1, s_2, \ldots$. At initialization, we'll have

\begin{align}
\texttt{dist}[s] & = 0 \\
\texttt{dist}[s_1] & = \infty\\
\texttt{dist}[s_2] & = \infty\\
\vdots
\end{align}

The edges $s\rightarrow s_1$, $s\rightarrow s_2$, etc, are tense because

\begin{align}
\underbrace{\texttt{dist}[s_k]}_{\infty} > \underbrace{\texttt{dist}[s]}_{0}+ \underbrace{\texttt{G}[s][s_k]}_{\color{red}{< \infty}}
\end{align}

We know that $\texttt{G}[s][s_k]<\infty$ because there is an edge from $s$ to $s_k$. These edges can be relaxed by updating their shortest path distances as

\begin{align}
\texttt{dist}[s_k] & =  \texttt{dist}[s] + \texttt{G}[s][s_k] & \\
            & =  \texttt{G}[s][s_k] &(\text{because } \texttt{dist}[s]=0)
\end{align}

This makes sense: the shortest path from $s$ to any of its adjacent vertices $s_k$ is the edge $s\rightarrow s_k$.



Once we establish the shortest paths from $s$ to its adjacent vertices $s_k$, we can relax any tense edges from $s_k$ forward, and so on. All we need is a place to keep track which vertices to explore next for tense edges emanating from them. If this sounds very similar to searching for vertices reachable from $s$, it is. And therefore, the mechanism is similar too: an array to hold the vertices to explore next.

```python
stack = [s]                         # Vertices to explore next.
while stack:                        # Same as while len(stack) > 0
  u = stack.pop()                   # Check edges out of u
  for v in range(len(G)):           # G is adjacency matrix
    if G[u][v] < G[0][0]:           # Diagonal element is infinity
      if dist[v] > dist[u]+G[u][v]: # Edge u->v is tense
        dist[v] > dist[u]+G[u][v]   # Relax it
        parent[v] = u               # Update path route
      stack.append(v)               # Plan to check edges out of v
```

As you may remember from earlier courses, the two most common ways to explore a graph are depth-first and breadth-first searches. Both techniques use an array to store which vertices to explore next. Their only difference is how they stage the data in that array. If data are staged in a first-in/first-out manner, the array acts as a queue. If data are staged in a last-in/first-out manner, the array acts as a stack. Queue-based explorations are breadth-first, stack-based are depth-first. Stack-based explorations can be implemented without an array as recursive programs that exploit the [program stack](https://en.wikipedia.org/wiki/Call_stack) in the computer's CPU.

When using an array to implement a depth-first search, stack operations in Python are straight forward: `list.append` adds a value at the back of an array and `list.pop()` removes a value from the back of the array. That's a LIFO staging.

A basic implementation of the single-source shortest path algorithm is shown below. Method `sssp(G,s)` takes the adjacency matrix `G` of a directed acyclic graph and computers the shortest paths to every vertex in the graph from a starting vertex `s`. You may test this code with the simple graph given in adjacency matrix `test` or write your own graph.

In [ ]:
def sssp(G,s):
  n = len(G)                            # Number of vertices
  infty = G[0][0]                       # No-edge value (infinity)
  dist = [ infty ] * n                  # Shortest path distances, initially infinite
  dist[s] = 0                           # Shortest path distance from source vertex
  parent = [ None ]*n                   # Parent of a vertex along the shortest path
  stack = [s]                           # Initialize exploration array with source vertex
  while stack:                          # Iterate as long as there are vertices to exploee
    u = stack.pop()                     # Next vertex to explore
    for v in range(n):                  # Consider all the neighbors of this vertex
      if G[u][v] < infty:               # v is a neighbor if the edge u -> v exists
        if dist[u] + G[u][v] < dist[v]: # Check to see if edge u -> v is tense
          dist[v] = dist[u] + G[u][v]   # If edge tense, relax it
          parent[v] = u                 # Adjust path to show we get to v through u
        stack.append(v)                 # Explore v next
  return dist, parent                   # Done

In [ ]:
_ = float('inf')

test = [
    [  _, 160,   _,   _, 140],
    [  _,   _,  50,   _,   _],
    [  _,   _,   _,  40,   _],
    [  _,   _,   _,   _,   _],
    [  _,   _,   _,  30,   _]
]

([0, 160, 210, 170, 140], [None, 0, 1, 4, 0])

## Which edges to explore?

So far, we explored the graph using a stack (or a queue) staging of the vertices to visit and look for tense edges. Another way to look for tense edges is to visit the vertices in their *topological order*. This is the order in which the vertices would be lined up if we stretched the graph between two special vertices: a *source* vertex and a *target* (or *sink*) vertex.

A source vertex has no incoming edges. A target vertex has no outgoing edges. In terms of adjacency matrix representation, a source vertex is a column of infinities and a target vertex is a row of infinities. In the earlier example

$$
\texttt{G} =
\begin{bmatrix}
\color{pink}\infty &    160 & \infty & \infty &    140 \\
\color{pink}\infty & \infty &     50 & \infty & \infty \\
\color{pink}\infty & \infty & \infty &     40 & \infty \\
\color{pink}\infty & \color{pink}\infty & \color{pink}\infty & \color{pink}\infty & \color{pink}\infty \\
\color{pink}\infty & \infty & \infty &     30 & \infty
\end{bmatrix}
$$

vertex 0 is the source vertex because $G[u][0]=\color{pink}\infty$ and vertex 3 is the target vertex because $G[3][v] =\color{pink}\infty$. From this, we can tell that a linear arrangement of $G$'s vertices will start with the source (vertex 0) and end with the target (vertex 3). In such an arrangement, *all edges point forward.*

## Topological order (sorting)

The topological order of a directed acyclic graph is the order in which we would be removing its source vertices. No vertices are removed in the process -- such foolishness would only destroy the graph. The process is purely fictional: *pretent to remove* one source vertex at a time.

What happens if the graph has only once source vertex? In this case, other vertices will become sources. And after we (pretent to) remove them, there will be new sources. Until all sources have been (hypothetically) removed and the remaining vertices in the (hypothetically) decimated graph are the target vertices.

The order in which source vertices are (hypothetically) removed, together with the last remaining target vertices is the topological order of the graph. The pseudocode for computing the topological order of a graph is as follows.

\begin{align}
& S \leftarrow \{\}  \\
& T \leftarrow () \\
& \text{for every } u \in V: \\
& \quad\text{if}\ d_\text{in}(u) = 0: \\
& \quad\quad S \cup u\\
& \text{while}\ |S| > 0: \\
& \quad u\leftarrow\ S.\texttt{pop} \\
& \quad \text{for every edge}\ u\rightarrow v: \\
& \quad\quad d_\text{in}(v)-1 \\
& \quad\quad\text{if}\ d_\text{in}(v) = 0: \\
& \quad\quad\quad S \cup u\\
& \quad\quad\quad T \cup u\\
& \text{return}\ T
\end{align}

In the pseudocode above $S$ is the set of source vertices identified during the (hypothetical) removal of vertices; $T$ is an ordered set, and; $d_\text{in}(j)$ is the in-degree of vertex $j$.

The in-degree of a vertex is the number of edges arriving at the vertex. In terms of an adjacency matrix $G$, the in-degree of a vertex is written as

\begin{align}
d_\text{in}(j) = \sum_{\substack{i=0\\ G_{ij} <\infty}}^{n-1}\mathbf{1}
\end{align}

In plain terms, the in-degree of a vertex $j$ is the number of values in the $j$-th column of the adjacency matrix that are less than infinity.

## Your assignment: part I

Given a weighted directed acyclic graph $G=(V,E,w)$ and a vertex $s$, find the shortest paths from $s$ to every other vertex in $G$ using the topological order to seek and relax tense edges. The results of your method should be similar to the results of `sssp` earlier.

This part is intentionally ambiguous. You have to compute single-source shortest paths in a weighted *dag*. Unlike the code given earlier in `sssp` you have to check for tense edges, following the topological order of the graph. Specifically, you'll need to explore the outgoing edges of each vertex in the the topological order.

The construction of the topological order was discussed in class on 10/4/24 and is described in pseudocode above.

## Your assignment: part II

For the *Ungrading* process in the course, please compare your assignments so far with the posted solutions. Identify any significant differences between your work and my solutions. Describe a few of them; no need to describe all, maybe 3. Discuss what you learned from them. Reflect on how this learning is different from what you may have covered in COMP 272 (if applicable).

Also discuss your class attendance and participation. This discussion should be an assessment of your engagement in the course; not a justification for absences or lack of participation. Use this discussion as an opportunity to plan your engagement for the rest of the term.

Also please take a few minutes to tell me how I can do a better job in the course -- you can use the anonymous feedback form for that if you wish.

Finally, based on your self-assessment, what is the course grade you would expect and why?

## How to submit

You can include all your work in the notebook with your solution to Part I of this assignment.

On Sakai, just copy the link to your Colab notebook (make sure that the notebook is shareable). For Part II you may also type things straight into Sakai or upload a PDF with your reflection.





# Assignment Part 1, Topological Order

In [ ]:
# function to get in degree of a given vertex, v
# Returns a list of the in_degree for the graph
def get_in_degree(G):
    # Initialize in_degree return array of len(num_vertices)
    in_degree = [0] * len(G)
    infinity = G[0][0]
    for v in range(len(G)):
        for u in range(len(G)):
            # If there is an edge from u to v, add to array
            if (G[u][v] < infinity):
                in_degree[v] += 1
    return in_degree

# Method to get the taopological order Given a graph, G, and a starting node, s
def topological(G, s):
    # Get the number of vertices
    n = len(G)
    # diagonal elements are always infinity
    infinity = G[0][0]
    # Initialize the stack
    S = []
    # Array that will store the topological order we will return
    T = []

    # Add all vertices with in-degree 0 to the stack, since they have no dependencies
    # and can be processed first
    in_degree = get_in_degree(G)
    # Add the elements that have an in_degree of 0, because they will be the
    # Next element to search for the topo order
    for u in range(n):
        if (in_degree[u] == 0):
            S.append(u)

    # Provess vertices until the stack is empty
    while(S):
        # Get the last vertice from the stack
        u = S.pop()
        # Add the new vertice to the topological return list
        T.append(u)

        # Iterate over the graph
        for v in range(n):
            # If there is an edge from u to v (G[u][v] is less than infinity),
            # it means v depends on u, so reduce the in-degree of v
            if G[u][v] < infinity:
                in_degree[v] -= 1

                # If the in-degree of v is now 0, add it to the stack for processing
                # and append it to the result array T
                if (in_degree[v] == 0):
                    S.append(v)

    return T

def sssp_topological(G, s):
    # Get number of vertices in graph
    n = len(G)

    # Get the topological ordering from the Graph
    topological_order = topological(G, s)

    # Initialize infinity, diagonal elements are always infinity
    infinity = G[0][0]

    # Initialize a distance array to track the shortest distance from the start vertex s
    # Set all distances to infinity because we don't know the shortest path yet
    dist = [infinity] * n

    # distance from start to itself is always 0
    dist[s] = 0

    # Loop over all of the nodes from the topo order
    for u in topological_order:
        # If we have already explored this edge
        if dist[u] != infinity:
            # Try and find a shorter path to this edge u
            for v in range(n):
                if G[u][v] < infinity:  # If there is an edge from u to v
                    # Relaxation step
                    if dist[v] > dist[u] + G[u][v]:
                        # We found a shorter path, so update
                        dist[v] = dist[u] + G[u][v]

    return dist

In [ ]:
# Sample use case

_ = float('inf')

test = [
    [  _, 160,   _,   _, 140],
    [  _,   _,  50,   _,   _],
    [  _,   _,   _,  40,   _],
    [  _,   _,   _,   _,   _],
    [  _,   _,   _,  30,   _]
]


sssp_topological(test, 0)

[0, 160, 210, 170, 140]

# Assignment Part 2 : Reflection

### Please refer to the sakai textbox